# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [71]:
!kubectl create namespace seldon

namespace/seldon created


In [72]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "gke_kubeflow-testing-213813_europe-west4-a_cluster-1" modified.


In [73]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [74]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [75]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [76]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sun Oct  7 12:39:01 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  1s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  1s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s


NOTES:
NOTES: TODO




In [77]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Sun Oct  7 12:39:04 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1beta1/Role
NAME          AGE
ambassador    1s
seldon-local  1s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-ambassador-778c58bf5d-7lkbm              0/2    ContainerCreating  0         0s
seldon-core-seldon-apiserver-6b8dbc978b-58jsx        0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-596d4674fd-bsl8b  0/1    ContainerCreating  0         0s
seldon-core-redis-8668565565-2fkh5                   0/1    ContainerCreating  0         0s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.39.240.4

In [78]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [79]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Serve Single Model

In [80]:
!helm install ../helm-charts/seldon-single-model --name mymodel

NAME:   mymodel
LAST DEPLOYED: Sun Oct  7 12:40:19 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  1s




In [81]:
!helm template ../helm-charts/seldon-single-model --name mymodel | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [82]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymodel-mymodel-classifier-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-classifier-0" successfully rolled out
deployment "mymodel-mymodel-svc-orch" successfully rolled out


### Get predictions

In [83]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [84]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "ughcvlducfb1brroaj6p88hedt",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08298236340068435]
    }
  }
}


#### gRPC Request

In [85]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "t7t456m66jup7bm8g590pldvtd"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08579566803827866
  }
}



In [86]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [87]:
!helm install ../helm-charts/seldon-abtest --name myabtest

NAME:   myabtest
LAST DEPLOYED: Sun Oct  7 12:41:36 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  0s




In [88]:
!helm template ../helm-charts/seldon-abtest --name myabtest | pygmentize -l json

---
# Source: seldon-abtest/templates/ab_test_1pod.json


---
# Source: seldon-abtest/templates/ab_test_2pods.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "myabtest"
    },
    "spec": {
	"name": "myabtest",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1M

In [90]:
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-classifier-1-0
!kubectl rollout status deploy/myabtest-abtest-svc-orch 

Waiting for deployment "myabtest-abtest-classifier-1-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-classifier-1-0" successfully rolled out
deployment "myabtest-abtest-svc-orch" successfully rolled out


### Get predictions

In [91]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [92]:
rest_request_ambassador("myabtest",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "38fk7g5fmoi8qms0k5lod8l8jk",
    "tags": {
    },
    "routing": {
      "myabtest": 1
    },
    "requestPath": {
      "myabtest": "",
      "classifier-2": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09353521837367941]
    }
  }
}


#### gRPC Request

In [93]:
grpc_request_ambassador("myabtest",API_AMBASSADOR)

meta {
  puid: "jbk8d40se4d9rbg5alpabu2tgq"
  routing {
    key: "myabtest"
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08099174567020175
  }
}



In [94]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [95]:
!helm install ../helm-charts/seldon-mab --name mymab

NAME:   mymab
LAST DEPLOYED: Sun Oct  7 12:43:32 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [96]:
!helm template ../helm-charts/seldon-mab --name mymab | pygmentize -l json

---
# Source: seldon-mab/templates/mab.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
	"labels": {
	    "app": "seldon"
	},
	"name": "mymab"
    },
    "spec": {
	"name": "mymab",
	"predictors": [
	    {
		"name": "abtest",
		"replicas": 1,
		"componentSpecs": [{
		    "spec": {
			"containers": [
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-1",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }],
			"terminationGracePeriodSeconds": 20
		    }},
	        {
		    "metadata":{
			"labels":{
			    "version":"v2"
			}
		    },    
			"spec":{
			    "containers":[
			    {
				"image": "seldonio/mock_classifier:1.0",
				"imagePullPolicy": "IfNotPresent",
				"name": "classifier-2",
				"resources": {
				    "requests": {
					"memory": "1Mi"
				    }
				}
			    }
			],
			"terminationGracePeriodSeconds": 20
			}


In [97]:
!kubectl rollout status deploy/mymab-abtest-classifier-1-0
!kubectl rollout status deploy/mymab-abtest-classifier-2-1
!kubectl rollout status deploy/mymab-abtest-eg-router-2 

Waiting for deployment "mymab-abtest-classifier-1-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-classifier-1-0" successfully rolled out
deployment "mymab-abtest-classifier-2-1" successfully rolled out
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "mymab-abtest-eg-router-2" rollout to finish: 0 of 1 updated replicas are available...

### Get predictions

In [98]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [99]:
rest_request_ambassador("mymab",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "lci2khaoammk4kigukd4e27gvm",
    "tags": {
    },
    "routing": {
      "eg-router": 0
    },
    "requestPath": {
      "eg-router": "seldonio/mab_epsilon_greedy:1.1",
      "classifier-1": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07165456095390127]
    }
  }
}


#### gRPC Request

In [100]:
grpc_request_ambassador("mymab",API_AMBASSADOR)

meta {
  puid: "9e4n8q1jkk3nofkvuinqal6dk7"
  routing {
    key: "eg-router"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10968943118218034
  }
}



In [101]:
!helm delete mymab --purge

release "mymab" deleted


## Serve Single Model with Outlier Detector

In [102]:
!helm install ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true 

NAME:   mymodel
LAST DEPLOYED: Sun Oct  7 12:54:51 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [103]:
!helm template ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true | pygmentize -l json

---
# Source: seldon-single-model/templates/model.json
{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mymodel"
    },
    "spec": {
        "name": "mymodel",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                        "terminationGracePeriodSecond

In [105]:
!kubectl rollout status deploy/mymodel-mymodel-classifier-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch
!kubectl rollout status deploy/mymodel-mymodel-outlier-detector-1

deployment "mymodel-mymodel-classifier-0" successfully rolled out
deployment "mymodel-mymodel-svc-orch" successfully rolled out
deployment "mymodel-mymodel-outlier-detector-1" successfully rolled out


### Get predictions

In [106]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [107]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "hcg7d669jnuvbun7j8au05pi9d",
    "tags": {
      "outlierScore": [0.0]
    },
    "routing": {
      "outlier-detector": -1
    },
    "requestPath": {
      "outlier-detector": "seldonio/outlier_mahalanobis:0.3",
      "classifier": "seldonio/mock_classifier:1.0"
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08697380989284384]
    }
  }
}


#### gRPC Request

In [108]:
grpc_request_ambassador("mymodel",API_AMBASSADOR)

meta {
  puid: "uotc4k27fdaofmmk9980sbseeq"
  tags {
    key: "outlierScore"
    value {
      list_value {
        values {
          number_value: 0.0
        }
      }
    }
  }
  routing {
    key: "outlier-detector"
    value: -1
  }
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "outlier-detector"
    value: "seldonio/outlier_mahalanobis:0.3"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06163667090761685
  }
}



In [109]:
!helm delete mymodel --purge

release "mymodel" deleted
